In [44]:
import pandas as pd
import numpy as np
import os
from datetime import date,timedelta

In [45]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\data'
asset_data = 'AssetData.csv'
asset_system = 'AssetStructureA&S.csv'
system_component = 'AssetStructureS&C.csv'
component_dutie = 'DutieStructureC&D.csv'
dutie_data = 'DutieStructureD&D.csv'
dutie_resource = 'DutieStructureD&R.csv'
wbs = 'WBS.csv'

ruteA_D = os.path.join(folder,asset_data)
ruteA_S = os.path.join(folder,asset_system)
ruteS_C = os.path.join(folder,system_component)
ruteC_D = os.path.join(folder,component_dutie)
ruteD_D = os.path.join(folder,dutie_data)
ruteD_R = os.path.join(folder,dutie_resource)
rute_wbs = os.path.join(folder,wbs)


In [59]:
dataA_D = pd.read_csv(ruteA_D, dtype = {'Cod sap':str})
dataA_S = pd.read_csv(ruteA_S, dtype = {'System':str})
dataS_C = pd.read_csv(ruteS_C)
dataC_D = pd.read_csv(ruteC_D)
dataD_D = pd.read_csv(ruteD_D, dtype = {'Specialist':str,'Amount Craft':int,'Time':float})
dataD_R = pd.read_csv(ruteD_R, dtype = {'Cod. Material':str})
data_wbs = pd.read_csv(rute_wbs)
dataD_D.loc[:,'Last date task done']=pd.to_datetime(dataD_D.loc[:,'Last date task done'],format='%d/%m/%Y')

In [60]:
## DATA OF DUTIES AND TASK INFORMATION

# Keeping just Mechanical and Predective specialist
dataD_D_MP = dataD_D.loc[(dataD_D.loc[:,'Specialist']=='TEC-MEC') | (dataD_D_1task.loc[:,'Specialist']=='TEC-PRD'),:].reset_index(drop = True)
dataD_D_M = dataD_D.loc[dataD_D.loc[:,'Specialist']=='TEC-MEC',:].reset_index(drop = True)
dataD_D_P = dataD_D.loc[dataD_D.loc[:,'Specialist']=='TEC-PRD',:].reset_index(drop = True)

# load the data to work on
#dataD_D_M.to_excel('dataD_D_M.xlsx',encoding='utf-8-sig')
#dataD_D_P.to_excel('dataD_D_P.xlsx',encoding='utf-8-sig')

# All the non Mechanical or Predective activities are remove
dataD_D_M.loc[:,'Man hour'] = dataD_D_M.loc[:,'Time']*dataD_D_M.loc[:,'Amount Craft']
dataD_D_P.loc[:,'Man hour'] = dataD_D_P.loc[:,'Time']*dataD_D_P.loc[:,'Amount Craft']

# All the conditional activities are remove (1T: first task: all activities with frequency 2T second task: all conditional activities)
dataD_D_M_1T = dataD_D_M.loc[dataD_D_M.loc[:,'Maintenance type'] != 'COND',:].reset_index(drop = True)
dataD_D_P_1T = dataD_D_P.loc[dataD_D_P.loc[:,'Maintenance type'] != 'COND',:].reset_index(drop = True)

dataD_D_2task = dataD_D_MP.loc[dataD_D_MP.loc[:,'Maintenance type'] == 'COND',:].reset_index(drop = True)

# Adding Estrategy to the activities
dataD_D_M_1T.loc[:,'Strategy'] = dataD_D_M_1T.loc[:,'Frequency'].astype(int).astype(str) + dataD_D_M_1T.loc[:,'Unidad Periodo']
dataD_D_P_1T.loc[:,'Strategy'] = dataD_D_P_1T.loc[:,'Frequency'].astype(int).astype(str) + dataD_D_P_1T.loc[:,'Unidad Periodo']


In [56]:
# DATA OF ASSET INFORMATION
# Removing Spanish information
asset_SP_EN = ['WBS_1_ES', 'WBS_2_ES', 'WBS_3_ES', 'WBS_4_ES']
dataA_D_EN = dataA_D.drop(asset_SP_EN, axis=1)
# All Johan&Jorge clearence
dataA_D_JJ = dataA_D_EN.loc[dataA_D.loc[:,'Responsable']=='Johan&Jorge',:].sort_values(by='Asset tag').reset_index(drop = True)

In [72]:
%%time
# Merging all datas csv 
# Asset&Data(Johan&Jorge) -> Asset&System -> System&Component -> Component&Duties -> 
# Duties&Data(Mechanical/Predective/Mechanical&Predective)
dataA_D_S = pd.merge(dataA_D_JJ,dataA_S,how = 'left',on = 'Asset tag')
dataA_S_C = pd.merge(dataA_D_S, dataS_C, how='left', on = 'key_system')
dataA_S_C_D = pd.merge(dataA_S_C, dataC_D, how='left', on='key_component')

dataASCD_M = pd.merge(dataA_S_C_D, dataD_D_M_1T, how = 'inner', on = 'key_dutie')
dataASCD_P = pd.merge(dataA_S_C_D, dataD_D_P_1T, how = 'inner', on = 'key_dutie')

# Saving the data
dataASCD_M.to_excel('dataASCD_M.xlsx',encoding='utf-8-sig', index=False)
dataASCD_P.to_excel('dataASCD_P.xlsx',encoding='utf-8-sig', index=False)


Wall time: 2.62 s


In [65]:
dataA_S_C_D.isnull().sum()

Asset tag             0
Name                 93
Cod sap             720
WBS_1_EN              0
WBS_2_EN              0
WBS_3_EN              0
WBS_4_EN              0
Responsable           0
Hour day              0
Criticidad            0
Criticidad Name       0
key_system            0
System             9702
key_component         0
Component             0
key_dutie             0
Primary task          0
dtype: int64

In [10]:
%%time
# Adding the pivot column to project
dataAllcompileM.loc[:,'pivot'] = dataAllcompileM.loc[:,'Asset tag'] + '-' + dataAllcompileM.loc[:,'Strategy'] 
# Saving all data compile Predective and Mechanical
dataAllcompileP.to_csv('data_all_prd.csv', encoding = 'utf-8-sig')
dataAllcompileP.to_excel('data_all_prd.xlsx', encoding = 'utf-8-sig')
dataAllcompileM.to_csv('data_all_mec.csv', encoding = 'utf-8-sig')
dataAllcompileM.to_excel('data_all_mec.xlsx', encoding = 'utf-8-sig')



NameError: name 'planDates' is not defined

In [12]:
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo','Strategy', 'Date task done']
projection = dataAllcompileM.loc[:,['Asset tag','Strategy','Frequency', 'Unidad Periodo','Date task done','Hour day']].groupby(['Asset tag','Strategy','Frequency', 'Unidad Periodo','Date task done','Hour day']).size().reset_index()
projection.drop(columns = {0},inplace =True)
projection.loc[:,'Date plan'] = projection.loc[:,'Date task done']
[row,column] = projection.shape

In [13]:
%%time
planDates = pd.DataFrame(columns = projection.columns)
projectionflag = projection

for m in range(row):
    while projectionflag.loc[m,'Date plan'] < date(2022,1,1):
        
        if projectionflag.loc[m,'Unidad Periodo'] == 'H':
            delta = timedelta(hours = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'S':
            delta = timedelta(weeks = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'M':
            delta = timedelta(weeks = 4*projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'A':
            delta = timedelta(weeks = 52*projectionflag.loc[m,'Frequency'])
        
        projectionflag.loc[m,'Date plan'] = projectionflag.loc[m,'Date plan'] + delta
        planDates = planDates.append(projectionflag.loc[m,:],ignore_index=True)

Wall time: 22.3 s


In [21]:
planDates.loc[:,'pivot'] = planDates.loc[:,'Asset tag'] + '-' + planDates.loc[:,'Strategy']

In [ ]:
dataAllcompileMplan = pd.merge(dataAllcompileM,planDates.loc[:,['pivot','Date plan']],how='left',on='pivot')
# save the datacompileMplan
dataAllcompileMplan.to_excel('dataAllcompileMplan.xlsx', encoding='utf-8-sig')

In [22]:
# DATA OF DATES AND PLANIFICATION
dataStrategy = dataAllcompileM.loc[:,['Frequency','Unidad Periodo']].groupby(['Frequency','Unidad Periodo']).size().reset_index()
dataStrategy.loc[:,0] = dataStrategy.loc[:,'Frequency'].astype(int).astype(str) + dataStrategy.loc[:,'Unidad Periodo']
dataStrategy.rename(columns = {0:'claveStrategy'},inplace =True)

In [16]:
# Data wrangling
filtertype = dataAllcompileM.loc[:,'Maintenance type'] == 'INSP'
filterunidad = dataAllcompileM.loc[:,'Unidad Periodo'] == 'H'
filterhour = dataAllcompileM.loc[:,'Frequency'] == 1400
dataAllcompileM.loc[filtertype & filterunidad,'Frequency_eq'] = (dataAllcompileM.loc[filtertype & filterunidad,'Frequency']/(24*7)).round(0)

In [17]:
dataAllcompileM.loc[filtertype & filterunidad,['Frequency','Frequency_eq']].groupby(['Frequency','Frequency_eq']).size()

Frequency  Frequency_eq
165.0      1.0             246
330.0      2.0             101
660.0      4.0             711
1000.0     6.0              24
1400.0     8.0               2
1500.0     9.0               7
2000.0     12.0            169
3000.0     18.0              4
4000.0     24.0             92
6000.0     36.0              1
8000.0     48.0             78
dtype: int64

In [18]:
# Combine activities based on similar columns
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo']
# '2253-PU-212'
rowfilter = dataAllcompileM.loc[:,'Asset tag'] == '2221-PU-201'
# dataAllcompile.loc[rowfilter,columnfilter]
datacombine = dataAllcompileM.loc[rowfilter,columnfilter].groupby(['key_system','Component','Maintenance type','Constraint','Frequency']).agg({'Primary task': lambda x: "\n".join(x)}).reset_index()

In [19]:
datacombine

,key_system,Component,Maintenance type,Constraint,Frequency,Primary task
0,2221-PU-201|01,Acople de alta,INSP,MAR,165.0,Inspección acople de alta de bomba
1,2221-PU-201|01,Acople de baja,INSP,MAR,165.0,Inspección acople de baja de bomba
2,2221-PU-201|01,Reductor de velocidad,INSP,MAR,165.0,Inspección de reductor de bomba\nInspec nivel ...
3,2221-PU-201|01,Reductor de velocidad,PREV,DET,4000.0,Cambio aceite de reductor de bomba
4,2221-PU-201|01,Sistema de enfriamiento,INSP,MAR,660.0,Insp general sistema enfriamiento
5,2221-PU-201|01,Sistema de enfriamiento,PREV,DET,4000.0,Cambio filtros aceite sist enfriam
6,2221-PU-201|01,Sistema de enfriamiento,PREV,DET,8000.0,Cambio bomba aceite sist enfriam
7,2221-PU-201|02,Carcasa,INSP,DET,660.0,Inspección de carcasa de bomba
8,2221-PU-201|02,Carcasa,INSP,MAR,165.0,Inspección stuffing box de carcasa
9,2221-PU-201|02,Carcasa,PREV,DET,1000.0,Cambio spare parte húmeda de bomba\nCambio de ...
